In [1]:
from glob import iglob, glob
import os
import sys
from python_speech_features import mfcc
import numpy as np
import random
import progressbar
import librosa
import json

In [2]:
#Corpus location
data_dir = '/home/minhhiu/MyProjects/Compressed Speech Data/full_command_data'

In [3]:
SPACE_TOKEN = '<space>'
SPACE_INDEX = 0
FIRST_INDEX = ord('a') - 1 

In [5]:
# create a list of all commands
train_f = open(os.path.join(data_dir,"train.txt"),"r")

#delete whatever was in the file
cmd_list_f = open(os.path.join(data_dir,"cmd_list.txt"),"w")
cmd_list_f.close()

cmd_list_f = open(os.path.join(data_dir,"cmd_list.txt"),"a")

cmd = train_f.readline()
prev_cmd = ''
while cmd != '':
    if cmd != prev_cmd:
        cmd_list_f.write(cmd)
        prev_cmd = cmd
    cmd = train_f.readline()
train_f.close()
cmd_list_f.close()

FileNotFoundError: [Errno 2] No such file or directory: '/home/minhhieu/My Projects/Compressed Speech Data/full_command_data/train.txt'

In [4]:
#create labels for characters
cmd_f = open(os.path.join(data_dir,"cmd_list.txt"),"r")

cmd = " "
char_list = []

while cmd != "":
    cmd = cmd_f.readline().lower()
    for char in cmd:
        if char not in char_list and char != " " and char != "\n":
            char_list.append(char)
print(char_list)
print(len(char_list))

cmd_f.close()

['đ', 'è', 'n', 'p', 'h', 'ò', 'g', 'k', 'á', 'c', 'b', 'ậ', 't', 'ủ', 'ắ', 'm', 'ế', 's', 'ơ', 'ố', 'i', 'ă', 'ộ', 'ả', 'ó', 'r', 'ử', 'a', 'ở', 'é', 'o', 'l', 'ê', 'x', 'u', 'y', 'ể', 'à', 'v', 'ỏ', 'í']
41


In [5]:
#character dictionary
encode_dic = {}
decode_dic = {}

encode_dic[SPACE_TOKEN] = 0
value = 1
for char in char_list:
    if not char:
        break
    
    encode_dic[char] = value
    value += 1

encode_dic['<blank>'] = value
print(encode_dic)

decode_dic = {v : k for k, v in encode_dic.items()}
# decode_dic[value] = '<blank>'

print(decode_dic)

label_dic = {}
label_dic["char_num"] = value
label_dic["encode"] = encode_dic
label_dic["decode"] = decode_dic

print(label_dic)

with open("encode_decode.json", "w", encoding="utf-8") as en_de_file:
    json.dump(label_dic, en_de_file, ensure_ascii=False)

{'<space>': 0, 'đ': 1, 'è': 2, 'n': 3, 'p': 4, 'h': 5, 'ò': 6, 'g': 7, 'k': 8, 'á': 9, 'c': 10, 'b': 11, 'ậ': 12, 't': 13, 'ủ': 14, 'ắ': 15, 'm': 16, 'ế': 17, 's': 18, 'ơ': 19, 'ố': 20, 'i': 21, 'ă': 22, 'ộ': 23, 'ả': 24, 'ó': 25, 'r': 26, 'ử': 27, 'a': 28, 'ở': 29, 'é': 30, 'o': 31, 'l': 32, 'ê': 33, 'x': 34, 'u': 35, 'y': 36, 'ể': 37, 'à': 38, 'v': 39, 'ỏ': 40, 'í': 41, '<blank>': 42}
{0: '<space>', 1: 'đ', 2: 'è', 3: 'n', 4: 'p', 5: 'h', 6: 'ò', 7: 'g', 8: 'k', 9: 'á', 10: 'c', 11: 'b', 12: 'ậ', 13: 't', 14: 'ủ', 15: 'ắ', 16: 'm', 17: 'ế', 18: 's', 19: 'ơ', 20: 'ố', 21: 'i', 22: 'ă', 23: 'ộ', 24: 'ả', 25: 'ó', 26: 'r', 27: 'ử', 28: 'a', 29: 'ở', 30: 'é', 31: 'o', 32: 'l', 33: 'ê', 34: 'x', 35: 'u', 36: 'y', 37: 'ể', 38: 'à', 39: 'v', 40: 'ỏ', 41: 'í', 42: '<blank>'}
{'char_num': 42, 'encode': {'<space>': 0, 'đ': 1, 'è': 2, 'n': 3, 'p': 4, 'h': 5, 'ò': 6, 'g': 7, 'k': 8, 'á': 9, 'c': 10, 'b': 11, 'ậ': 12, 't': 13, 'ủ': 14, 'ắ': 15, 'm': 16, 'ế': 17, 's': 18, 'ơ': 19, 'ố': 20, 'i': 21

In [ ]:
target_txt = 'Đèn phòng ngủ chuyển màu xanh'
original = ' '.join(target_txt.strip().lower().split(' ')).replace('.', '').replace('?', '').replace(',', '').replace("'", '').replace('!', '').replace('-', '').replace('\t', '').replace(')', '').replace('"', '')
targets = original.replace(' ', '  ')
targets = targets.split(' ')
print(targets)
    
# Adding blank label
targets = np.hstack([SPACE_TOKEN if x == '' else list(x) for x in targets])
print(targets)

# Transform char into index
targets = np.asarray([encode_dic[x] for x in targets])
print(targets)

In [ ]:
#create a text file corresponding to every single records
cmd_list_f = open(os.path.join(data_dir,"cmd_list.txt"), "r")

directory = 1

while directory <= 71:
    text = cmd_list_f.readline()
    
    wavs = [f for f in os.listdir(os.path.join(data_dir, str(directory))) 
            if os.path.isfile(os.path.join(data_dir, str(directory), f)) 
            and f.endswith('.wav')]
    for wav in wavs:
        wavpath = os.path.join(data_dir, str(directory), wav)
        wavpath = wavpath.replace('wav','txt')
        
        text_f = open(os.path.join(wavpath), "w")
        text_f.write(text)
        text_f.close()
    directory = directory + 1
cmd_list_f.close()


In [6]:
def data_split(folder, partition_dict=None, seed=78):
    """Split VCTK data into train, dev, test sets.
        Args:
            folder: the folder path to the data (string)
            partition_dict: dictionary for train/dev/test split (default 0.8/0.1/0.1)
        Returns:
            None
    """
    if partition_dict is None:
        partition_dict = {'train':0.8, 'dev':0.1, 'test':0.1}
    assert sum(partition_dict.values()) == 1
    speaker_folders = glob(os.path.join(folder,'*'))
    for speaker_folder in speaker_folders:
        #print(speaker_folder)
        wav_files = glob(os.path.join(speaker_folder, '*.wav' ))
        #print(len(wav_files))
        random.seed(seed)
        random.shuffle(wav_files)
        quantities = [(name, round(ratio*len(wav_files))) for (name, ratio) in partition_dict.items()]
        for name, quantity in quantities:
            #print(quantity)
            for _ in range(quantity):
                try:
                    audio = wav_files.pop()
                    new_path_wav = os.path.join(folder, name, 'wav', speaker_folder.split('/')[-1], os.path.basename(audio))
                    os.renames(audio, new_path_wav)
                    old_path_txt = audio.replace("wav","txt").replace("wav","txt")
                    new_path_txt = new_path_wav.replace("wav","txt").replace("wav","txt")
                    os.renames(old_path_txt, new_path_txt)
                except IndexError as e:
                    pass

def find_files(directory, pattern='**/*.wav'):
    """Recursively finds all files matching the pattern."""
    return sorted(iglob(os.path.join(directory, pattern), recursive=True))

def read_audio_from_filename(filename, sample_rate):
    """Load a wav file and transpose the array."""
    audio, _ = librosa.load(filename, sr=sample_rate, mono=True)
    audio = audio.reshape(-1, 1)
    return audio

def convert_txt_index(target_txt):
    """Turn text into index."""
    original = ' '.join(target_txt.strip().lower().split(' ')).replace('.', '').replace('?', '').replace(',', '').replace("'", '').replace('!', '').replace('-', '').replace('\t', '').replace(')', '').replace('"', '')
    targets = original.replace(' ', '  ')
    targets = targets.split(' ')
    
    # Adding blank label
    targets = np.hstack([SPACE_TOKEN if x == '' else list(x) for x in targets])
    # Transform char into index
    targets = np.asarray([encode_dic[x] for x in targets])
    return targets, original

def return_txt_path(wav_path):
    """Return the corresponding txt location for VCTK data set."""
    return wav_path.replace("wav","txt").replace("wav","txt")

def find_speaker_ID(wav_path):
    """Find speaker ID from the path of a wav file."""
    return (wav_path.split('.')[0].split('/')[-1] + '_' +
            wav_path.split('.')[0].split('/')[-2])

def pack_data_npz(DIR, input_mfcc, target, speaker_wav_ID, original):
    """Pickle data into npz files."""
    np.savez(os.path.join(DIR, speaker_wav_ID),\
             data_in=input_mfcc, target=target, seq_len=np.array([len(input_mfcc)]), original=np.array([original]))

def convert_wav_mfcc(file, fs):
    """Turn raw audio data into MFCC with sample rate=fs."""
    inputs = mfcc(read_audio_from_filename(file, fs),samplerate=16000,winlen=0.025,winstep=0.01,numcep=39,
                 nfilt=40)
    return inputs

In [ ]:
#Split VCTK data into trian/test/dev set
data_split(data_dir)

In [7]:
#find mean, and varience of training data for normalization
directory = data_dir + '/train/wav'

bar = progressbar.ProgressBar()
n =0
sum_mfcc = np.zeros(39) 
sumsq_mfcc = np.zeros(39)
total_len = 0
for file in bar(find_files(directory, pattern='**/*.wav')): 
    audio = mfcc(read_audio_from_filename(file, 16000),samplerate=16000,winlen=0.025,winstep=0.01,numcep=39,
                 nfilt=40)

    sum_mfcc += np.sum(audio, axis = 0)
    sumsq_mfcc +=np.sum(audio*audio, axis = 0)
    total_len += len(audio)
    n += 1

m = sum_mfcc/total_len
v = sumsq_mfcc/(total_len-1) - m*m
s = np.sqrt(v)

print(m)
print(v)
print(s)


100% |########################################################################|


[ -7.11787461  -3.80257675  -5.1892211   -6.10649606 -14.93026671
  -6.50852211 -11.0820522  -11.38528159  -5.4359234  -11.92452621
  -7.35473919  -8.55734981 -13.01601318  -3.9352292   -8.98238838
  -4.68796679  -1.95915871  -1.73154274  -2.08739357  -0.03456929
  -0.53198038   0.01748119  -0.11119716   0.09039443   0.353582
  -0.09068641   0.32179263  -0.5481161   -0.26260662  -0.2551881
  -0.05354107   0.36378776   0.36751137  -0.67487943   0.03250879
  -0.79729677  -0.62674145  -0.34291148  -0.49919057]
[4.80709436e+01 2.73225564e+02 2.56144587e+02 2.82365340e+02
 3.64473904e+02 3.50754526e+02 2.94261552e+02 2.77978943e+02
 2.59434337e+02 2.72846929e+02 2.23980478e+02 2.38807094e+02
 2.40063544e+02 1.45434149e+02 1.30000553e+02 8.78272953e+01
 7.08331217e+01 4.77805934e+01 2.90516462e+01 1.77374929e+01
 9.81691185e+00 3.53698520e+00 4.37489065e-01 1.28564162e-01
 1.73595741e+00 4.84757899e+00 8.52678182e+00 1.32094524e+01
 1.59793449e+01 1.90119549e+01 2.20582108e+01 2.35460995e+01

In [15]:
print(total_len)

490133


In [8]:
Train_DIR = data_dir + '/MFCC_39_16khz/train/'
directory = data_dir + '/train/'

In [9]:
# Pickle Training data

if not os.path.exists(Train_DIR):
    os.makedirs(Train_DIR)
    
bar = progressbar.ProgressBar()
for wav_path in bar(find_files(directory, pattern='**/*.wav')):
    #print(wav_path)
    speaker_ID = find_speaker_ID(wav_path)
    #print(speaker_ID)
    txt_path = return_txt_path(wav_path)
    target, original = convert_txt_index(open(txt_path).read().strip())
    inputs = convert_wav_mfcc(wav_path, 16000)
    normalize_inputs = (inputs - m)/s
    pack_data_npz(Train_DIR, normalize_inputs, target, speaker_ID, original)


100% |########################################################################|


In [10]:
# Pickle Dev data

DIR = data_dir + '/MFCC_39_16khz/dev/'
directory = data_dir + '/val/'

if not os.path.exists(DIR):
    os.makedirs(DIR)
    
bar = progressbar.ProgressBar()
for wav_path in bar(find_files(directory, pattern='**/*.wav')):
    #print(wav_path)
    speaker_ID = find_speaker_ID(wav_path)
    #print(speaker_ID)
    txt_path = return_txt_path(wav_path)
    target, original = convert_txt_index(open(txt_path).read().strip())
    inputs = convert_wav_mfcc(wav_path, 16000)
    normalize_inputs = (inputs - m)/s
    pack_data_npz(DIR, normalize_inputs, target, speaker_ID, original)

100% |########################################################################|


In [11]:
# Pickle Test data

DIR = data_dir + '/MFCC_39_16khz/test/'
directory = data_dir + '/test/'

if not os.path.exists(DIR):
    os.makedirs(DIR)
    
bar = progressbar.ProgressBar()
for wav_path in bar(find_files(directory, pattern='**/*.wav')):
    #print(wav_path)
    speaker_ID = find_speaker_ID(wav_path)
    #print(speaker_ID)
    txt_path = return_txt_path(wav_path)
    target, original = convert_txt_index(open(txt_path).read().strip())
    inputs = convert_wav_mfcc(wav_path, 16000)
    
    normalize_inputs = (inputs - m)/s
    pack_data_npz(DIR, normalize_inputs, target, speaker_ID, original)

100% |########################################################################|


In [1]:
%reload_ext tensorboard
%tensorboard --logdir '/home/minhhiu/MyProjects/speech/logs'

In [4]:
DIR = data_dir + '/MFCC_39_16khz/test/rec_8_1.npz'
data = np.load(DIR)
print(data['data_in'])

[[-9.53987793 -1.269448    1.67310601 ... -2.27360753 -0.4295214
  -3.69245003]
 [-9.8267141  -1.28899542  1.87876153 ... -0.34186347 -0.09316157
  -4.08470592]
 [-9.8248009  -3.34290005 -0.75952223 ... -0.49052357 -0.16833934
   0.20581648]
 ...
 [-8.57498707 -7.17791755 10.58529722 ...  0.12683245  6.34203963
  -0.48419089]
 [-8.53691756 -6.65650265 15.18826144 ... -1.8499269   1.57116769
  -2.23493946]
 [-8.19465177 -7.62449687  5.21408897 ... -3.14172332  0.31766861
  -3.21747218]]
